In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dogs-vs-cats/test1.zip
/kaggle/input/dogs-vs-cats/train.zip
/kaggle/input/dogs-vs-cats/sampleSubmission.csv


In [2]:
train_path = "../input/dogs-vs-cats/test1.zip"
test_path = "../input/dogs-vs-cats/train.zip"

files = "/kaggle/working/"

import zipfile

with zipfile.ZipFile(train_path, 'r') as zipp:
    zipp.extractall(files)
    
with zipfile.ZipFile(test_path, 'r') as zipp:
    zipp.extractall(files)

In [3]:
image_dir = "../working/train/"

filenames = os.listdir(image_dir)

labels = [x.split(".")[0] for x in filenames]

data = pd.DataFrame({
     "filename": filenames, 
     "label": labels})
data.head()

image_dir = "../working/test1/"

filenames = os.listdir(image_dir)

labeltest = [x.split(".")[0] for x in filenames]

datatest = pd.DataFrame({
    "id": labeltest,
     "filename": filenames})
datatest.head()

,id,filename
0,10415,10415.jpg
1,4962,4962.jpg
2,1315,1315.jpg
3,11639,11639.jpg
4,12003,12003.jpg


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.image import imread
plt.figure(figsize=(20,20)) # specifying the overall grid size
plt.subplots_adjust(hspace=0.4)


for i in range(10):
    
    plt.subplot(1,10,i+1)    # the number of images in the grid is 10*10 (100)
    filename = 'train/' + 'cat.' + str(i) + '.jpg'
    image = imread(filename)
    plt.imshow(image)
    plt.title('Cat',fontsize=12)
    plt.axis('off')

plt.show()

In [4]:
from sklearn.model_selection import train_test_split
X=data['filename']
y=data['label']
X_train,X_val=train_test_split(data,random_state=42,stratify=y,test_size=0.2)

In [6]:
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen=ImageDataGenerator(rescale=1./255.,rotation_range=40,width_shift_range=0.2,height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
test_datagen=ImageDataGenerator(rescale=1./255.)
train_idg = datagen.flow_from_dataframe(X_train, "train/", x_col= "filename", y_col= "label",
                                    batch_size = 64,
                                    target_size=(150,150),class_mode='binary')
val_idg = test_datagen.flow_from_dataframe(X_val,"train/", x_col= "filename", y_col= "label",
                                    batch_size = 64,
                                    target_size=(150,150),class_mode='binary')
test_gen = test_datagen.flow_from_dataframe(datatest,"test1/",x_col= "filename", y_col= None, 
                                          class_mode=None,
                                          batch_size = 64,
                                          target_size = (150,150),
                                          shuffle = False
                                         )
test_gen

2024-07-01 16:06:54.448754: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-01 16:06:54.448853: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-01 16:06:54.634920: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.
Found 12500 validated image filenames.


In [7]:
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, InputLayer, Dropout, GlobalAveragePooling2D
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model

In [8]:
from tensorflow.keras.applications.vgg16 import VGG16
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                 input_shape=(150,150,3))
conv_base.trainable=False

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [9]:
x = conv_base.output
x = GlobalAveragePooling2D()(x)
x=Dropout(0.25)(x)
x = Dense(1024, activation='relu')(x)
x=Dropout(0.25)(x)
x = Dense(1024, activation='relu')(x)
x=Dropout(0.25)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=conv_base.input, outputs=predictions)

In [10]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    metrics=['accuracy'],
    loss='binary_crossentropy'
)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 150, 150, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 150, 150, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 75, 75, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 75, 75, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 75, 75, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 37, 37, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 37, 37, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 37, 37, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 37, 37, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 18, 18, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 18, 18, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 18, 18, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 18, 18, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 9, 9, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 4, 4, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 16,290,625 (62.14 MB)

 Trainable params: 1,575,937 (6.01 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [11]:
model.fit(train_idg,epochs=10,batch_size=64,validation_data=val_idg)

Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  1/313 ━━━━━━━━━━━━━━━━━━━━ 3:08:03 36s/step - accuracy: 0.5469 - loss: 0.7217

I0000 00:00:1719850102.309955     151 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1719850102.330448     151 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


220/313 ━━━━━━━━━━━━━━━━━━━━ 42s 457ms/step - accuracy: 0.5402 - loss: 0.6907

W0000 00:00:1719850202.510479     152 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


313/313 ━━━━━━━━━━━━━━━━━━━━ 185s 476ms/step - accuracy: 0.5530 - loss: 0.6848 - val_accuracy: 0.7648 - val_loss: 0.5850
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 126s 395ms/step - accuracy: 0.6753 - loss: 0.6186 - val_accuracy: 0.7898 - val_loss: 0.5035
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 126s 395ms/step - accuracy: 0.7187 - loss: 0.5684 - val_accuracy: 0.8050 - val_loss: 0.4495
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 126s 394ms/step - accuracy: 0.7427 - loss: 0.5295 - val_accuracy: 0.8140 - val_loss: 0.4108
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 125s 391ms/step - accuracy: 0.7606 - loss: 0.5033 - val_accuracy: 0.8216 - val_loss: 0.3870
Epoch 6/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 126s 393ms/step - accuracy: 0.7662 - loss: 0.4883 - val_accuracy: 0.8222 - val_loss: 0.3781
Epoch 7/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 125s 391ms/step - accuracy: 0.7750 - loss: 0.4705 - val_accuracy: 0.8370 - val_loss: 0.3580
Epoch 8/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 125s 392ms/step - accuracy: 0.7757 - loss: 0.46

In [12]:
test_pred = model.predict(test_gen)
test_pred

196/196 ━━━━━━━━━━━━━━━━━━━━ 35s 178ms/step


W0000 00:00:1719851624.264889     154 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


array([[0.9665265 ],
       [0.9943074 ],
       [0.9700998 ],
       ...,
       [0.019491  ],
       [0.17271854],
       [0.99453104]], dtype=float32)

In [13]:
y_pred_submit = (test_pred >= 0.5).astype(float)
y_test_submit=y_pred_submit.reshape(1,-1).squeeze()
y_test_submit

array([1., 1., 1., ..., 0., 0., 1.])

In [14]:
result_id=datatest['id']
result = pd.DataFrame({
    'id': result_id,
    'label': y_test_submit
})

In [15]:
result.to_csv("submission.csv", index=False)